# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 22 2023 9:20AM,257913,10,LUPLUP0017,"Lupin Research, Inc.",Released
1,Feb 22 2023 9:16AM,257912,10,SO94548,"Senseonics, Incorporated",Released
2,Feb 22 2023 9:16AM,257911,10,9191734,"Digital Brands, LLC",Released
3,Feb 22 2023 9:10AM,257910,10,MSP218782,"Methapharm, Inc.",Released
4,Feb 22 2023 9:10AM,257910,10,MSP218747,Methapharm-G,Released
5,Feb 22 2023 9:08AM,257909,10,37734,"Digital Brands, LLC",Released
6,Feb 22 2023 9:08AM,257909,10,37733,"Digital Brands, LLC",Released
7,Feb 22 2023 9:08AM,257909,10,37735,"Digital Brands, LLC",Released
8,Feb 22 2023 8:53AM,257908,10,Clear-9191233,"ClearSpec, LLC",Released
9,Feb 22 2023 8:51AM,257906,19,MSP218766,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
40,257909,Released,3
41,257910,Released,2
42,257911,Released,1
43,257912,Released,1
44,257913,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
257909,NaN,NaN,3.0
257910,NaN,NaN,2.0
257911,NaN,NaN,1.0
257912,NaN,NaN,1.0
257913,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257811,0.0,0.0,12.0
257817,0.0,0.0,8.0
257825,0.0,0.0,1.0
257832,4.0,0.0,4.0
257837,1.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257811,0,0,12
257817,0,0,8
257825,0,0,1
257832,4,0,4
257837,1,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257811,0,0,12
1,257817,0,0,8
2,257825,0,0,1
3,257832,4,0,4
4,257837,1,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257811,,,12
1,257817,,,8
2,257825,,,1
3,257832,4,,4
4,257837,1,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 22 2023 9:20AM,257913,10,"Lupin Research, Inc."
1,Feb 22 2023 9:16AM,257912,10,"Senseonics, Incorporated"
2,Feb 22 2023 9:16AM,257911,10,"Digital Brands, LLC"
3,Feb 22 2023 9:10AM,257910,10,"Methapharm, Inc."
4,Feb 22 2023 9:10AM,257910,10,Methapharm-G
5,Feb 22 2023 9:08AM,257909,10,"Digital Brands, LLC"
8,Feb 22 2023 8:53AM,257908,10,"ClearSpec, LLC"
9,Feb 22 2023 8:51AM,257906,19,"Methapharm, Inc."
10,Feb 22 2023 8:50AM,257907,10,"Snap Medical Industries, LLC"
12,Feb 22 2023 8:48AM,257905,12,"Uniderm USA, Inc"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 22 2023 9:20AM,257913,10,"Lupin Research, Inc.",,,1
1,Feb 22 2023 9:16AM,257912,10,"Senseonics, Incorporated",,,1
2,Feb 22 2023 9:16AM,257911,10,"Digital Brands, LLC",,,1
3,Feb 22 2023 9:10AM,257910,10,"Methapharm, Inc.",,,2
4,Feb 22 2023 9:10AM,257910,10,Methapharm-G,,,2
5,Feb 22 2023 9:08AM,257909,10,"Digital Brands, LLC",,,3
6,Feb 22 2023 8:53AM,257908,10,"ClearSpec, LLC",,,1
7,Feb 22 2023 8:51AM,257906,19,"Methapharm, Inc.",,,1
8,Feb 22 2023 8:50AM,257907,10,"Snap Medical Industries, LLC",,,2
9,Feb 22 2023 8:48AM,257905,12,"Uniderm USA, Inc",,,6


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 22 2023 9:20AM,257913,10,"Lupin Research, Inc.",1,,
1,Feb 22 2023 9:16AM,257912,10,"Senseonics, Incorporated",1,,
2,Feb 22 2023 9:16AM,257911,10,"Digital Brands, LLC",1,,
3,Feb 22 2023 9:10AM,257910,10,"Methapharm, Inc.",2,,
4,Feb 22 2023 9:10AM,257910,10,Methapharm-G,2,,
5,Feb 22 2023 9:08AM,257909,10,"Digital Brands, LLC",3,,
6,Feb 22 2023 8:53AM,257908,10,"ClearSpec, LLC",1,,
7,Feb 22 2023 8:51AM,257906,19,"Methapharm, Inc.",1,,
8,Feb 22 2023 8:50AM,257907,10,"Snap Medical Industries, LLC",2,,
9,Feb 22 2023 8:48AM,257905,12,"Uniderm USA, Inc",6,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 22 2023 9:20AM,257913,10,"Lupin Research, Inc.",1,,
1,Feb 22 2023 9:16AM,257912,10,"Senseonics, Incorporated",1,,
2,Feb 22 2023 9:16AM,257911,10,"Digital Brands, LLC",1,,
3,Feb 22 2023 9:10AM,257910,10,"Methapharm, Inc.",2,,
4,Feb 22 2023 9:10AM,257910,10,Methapharm-G,2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 22 2023 9:20AM,257913,10,"Lupin Research, Inc.",1.0,NaN,NaN
1,Feb 22 2023 9:16AM,257912,10,"Senseonics, Incorporated",1.0,NaN,NaN
2,Feb 22 2023 9:16AM,257911,10,"Digital Brands, LLC",1.0,NaN,NaN
3,Feb 22 2023 9:10AM,257910,10,"Methapharm, Inc.",2.0,NaN,NaN
4,Feb 22 2023 9:10AM,257910,10,Methapharm-G,2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 22 2023 9:20AM,257913,10,"Lupin Research, Inc.",1.0,0.0,0.0
1,Feb 22 2023 9:16AM,257912,10,"Senseonics, Incorporated",1.0,0.0,0.0
2,Feb 22 2023 9:16AM,257911,10,"Digital Brands, LLC",1.0,0.0,0.0
3,Feb 22 2023 9:10AM,257910,10,"Methapharm, Inc.",2.0,0.0,0.0
4,Feb 22 2023 9:10AM,257910,10,Methapharm-G,2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,6962586,154.0,5.0,18.0
12,1289506,10.0,0.0,0.0
19,1547276,34.0,3.0,0.0
21,515718,2.0,0.0,0.0
22,515747,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,6962586,154.0,5.0,18.0
1,12,1289506,10.0,0.0,0.0
2,19,1547276,34.0,3.0,0.0
3,21,515718,2.0,0.0,0.0
4,22,515747,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,154.0,5.0,18.0
1,12,10.0,0.0,0.0
2,19,34.0,3.0,0.0
3,21,2.0,0.0,0.0
4,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,154.0
1,12,Released,10.0
2,19,Released,34.0
3,21,Released,2.0
4,22,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19,21,22
Status,,,,,
Completed,18.0,0.0,0.0,0.0,0.0
Executing,5.0,0.0,3.0,0.0,0.0
Released,154.0,10.0,34.0,2.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19,21,22
0,Completed,18.0,0.0,0.0,0.0,0.0
1,Executing,5.0,0.0,3.0,0.0,0.0
2,Released,154.0,10.0,34.0,2.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19,21,22
0,Completed,18.0,0.0,0.0,0.0,0.0
1,Executing,5.0,0.0,3.0,0.0,0.0
2,Released,154.0,10.0,34.0,2.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()